In [1]:
import torch
import numpy as np

In [2]:
# view or reshape
# 每个view都有着他自己的意义的！真的很对！
# 另外view里面的 prod(rank) 必须和 prod(origin_rank)相等
a = torch.rand(4, 1, 28, 28)
print(a.shape)
b = a.view(4, 28 * 28)
print(b.shape)
c = a.view(4 * 28, 28) # 此时我相当于只关心每行的元素
print(c.shape)
d = a.view(4 * 1, 28, 28) # 此时相当于只关心所有的feature map

# 注意这里的 意义 的含义
e = d.view(4, 28, 28, 1) # 在操作上没问题的，但是意义错了，造成了数据错乱！！有逻辑错误。

torch.Size([4, 1, 28, 28])
torch.Size([4, 784])
torch.Size([112, 28])


In [9]:
# squeeze/unsqueeze
# 注意unsqueeze正索引和负索引的意义是不一样的，正索引表示在 ‘前’插入， 负索引表示在 ‘后’插入
# 所以索引的范围：比如rank=3，那么索引的范围就是 [-3, 2] 
# unsqueeze
b = a.unsqueeze(0)
print(b.shape) # [1, 4, 1, 28, 28]
c = a.unsqueeze(-1)
print(c.shape) # [4, 1, 28, 28, 1]  # 之后
d = a.unsqueeze(3)
print(d.shape) # [4, 1, 28, 1, 28]
e = a.unsqueeze(-5)
print(e.shape) # [1, 4, 1, 28, 28]
print('-' * 30)

# squeeze
# 注意squeeze在传入非1的维度的时候，什么也不做，并不会报错
f = e.squeeze() # 任何索引都不传，能消的全消
print(f.shape)
g = e.squeeze(0)
print(g.shape)
h = g.squeeze(1)
print(h.shape)

torch.Size([1, 4, 1, 28, 28])
torch.Size([4, 1, 28, 28, 1])
torch.Size([4, 1, 28, 1, 28])
torch.Size([1, 4, 1, 28, 28])
------------------------------
torch.Size([4, 28, 28])
torch.Size([4, 1, 28, 28])
torch.Size([4, 28, 28])


In [17]:
# expand/repeat
a = torch.randn(1, 3, 4, 4)
b = torch.ones(3)
b_unsqueeze = b.unsqueeze(0).unsqueeze(-1).unsqueeze(-1)
# expand
# 仅仅是对数据进行复制，使得它们能相加
# 注意只能对rank中值为1的维度进行复制，其他的数会报错
# 另外如果对当前维度不像expand，写成-1即可
b_unsqueeze_expand = b_unsqueeze.expand(1, 3, 4, 4) # [1, 3, 1, 1] -> [1, 3, 4, 4]
c = a + b_unsqueeze_expand
print(c.shape)
c_expand = c.expand(4, 3, -1, -1) # [4, 3, 4, 4]
print(c_expand.shape)

print('-' * 30)
# 一般不建议使用repeat函数进行数据复制，因为它会申请新的内存去存数据，把当前数据复制到别的更大的地方
# repeat中的参数的含义表示 该维度的数据需要赋值几次
b_unsqueeze_repeat = b_unsqueeze.repeat(1, 3, 4, 4) # [1, 9, 4, 4]，此时的参数和expand相同
print(b_unsqueeze_repeat.shape)
# 正确的用法
b_unsqueeze_repeat = b_unsqueeze.repeat(1, 1, 4, 4)
print(b_unsqueeze_repeat.shape) # [1, 3, 4, 4]

torch.Size([1, 3, 4, 4])
torch.Size([4, 3, 4, 4])
------------------------------
torch.Size([1, 9, 4, 4])
torch.Size([1, 3, 4, 4])


In [18]:
# .t
# 转置操作只使用于2D Tensor，任何别的维度的Tensor都会报错
a = torch.randn(2, 3)
print(a)
b = a.t()
print(b)

tensor([[-0.7009, -0.6653, -1.0090],
        [-0.4819, -2.3559, -0.6941]])
tensor([[-0.7009, -0.4819],
        [-0.6653, -2.3559],
        [-1.0090, -0.6941]])


In [28]:
# transpose操作
# 更加通用的转置操作，不仅仅是2D Tensor.
# 注意transpose操作后，Tensor有可能变得不是在一个连续的内存空间中了，调用.view()操作就会报错，因此一般的，在
# 调用transpose操作后都会用.contiguous()来让Tensor变得在同一块连续的内存空间中
a = torch.randn(4, 3, 32, 32, dtype=torch.float)
a1 = a.transpose(1, 3) # 将rank1和rank3进行交换，[4, 32, 32, 3]
print(a1.shape)
a2 = a1.contiguous() # 变得连续，注意这步操作非常重要，否则会报错
a3 = a2.view(4, 3 * 32 * 32)
a4 = a3.view(4, 32, 32, 3)
a5 = a4.transpose(1, 3)  # 兜了一圈，哈哈，一定要注意那个问题：就是view的逻辑意义！不要丢失信息
# validation
# torch.eq 判断两个Tensor的值是否都相等，element-wise的操作
# torch.all 如果全为True，或者Tensor中全部元素都不为0，则范围tensor(1, dtype=torch.uint8)
print(torch.all(torch.eq(a5, a))) 

torch.Size([4, 32, 32, 3])
tensor(1, dtype=torch.uint8)


In [33]:
# permute
a = torch.Tensor(4, 3, 32, 32) # [N, C, H, W]
print(a.shape)
b = a.permute([0, 2, 3, 1]) # [N, H, W, C]
print(b.shape)
b_reshape = b.contiguous().reshape(4, -1) # 一定要注意permute和transpose方法可能会让Tensor的内存空间变得不连续，要调用contiguous方法

torch.Size([4, 3, 32, 32])
torch.Size([4, 32, 32, 3])
